In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
!pip install -q torchinfo
from torchinfo import summary

In [3]:
## AlexNet change Sigmoid function to Relu: why? 
## -- Relu more esay to compute compare to sigmoid (complex exponiential computation)
## -- if initialize weight close to 0 or 1, sigmoid will have 0 gradient. The model will be hard to update through backpropagation

class AlexNet(nn.Module):
    def __init__(self):
        # input shape : (3,224,224)
        super().__init__()
        self.conv1 = nn.Conv2d(3,96,11,stride = 4) 
        self.pool = nn.MaxPool2d(3, 2)          
        self.conv2 = nn.Conv2d(96, 256, 5, padding = 2)   
        self.conv3 = nn.Conv2d(256, 384,3,  padding = 1)   
        self.conv4 = nn.Conv2d(384, 384,3, padding = 1)  
        self.conv5 = nn.Conv2d(384, 256, 3, padding = 1)  

        self.fc1 = nn.Linear(6400, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 1000)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.pool(self.relu(self.conv5(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


net = AlexNet() 

In [7]:
summary(model=net, 
        input_size=(1, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
AlexNet (AlexNet)                        [1, 3, 224, 224]     [1, 1000]            --                   True
├─Conv2d (conv1)                         [1, 3, 224, 224]     [1, 96, 54, 54]      34,944               True
├─ReLU (relu)                            [1, 96, 54, 54]      [1, 96, 54, 54]      --                   --
├─MaxPool2d (pool)                       [1, 96, 54, 54]      [1, 96, 26, 26]      --                   --
├─Conv2d (conv2)                         [1, 96, 26, 26]      [1, 256, 26, 26]     614,656              True
├─ReLU (relu)                            [1, 256, 26, 26]     [1, 256, 26, 26]     --                   --
├─MaxPool2d (pool)                       [1, 256, 26, 26]     [1, 256, 12, 12]     --                   --
├─Conv2d (conv3)                         [1, 256, 12, 12]     [1, 384, 12, 12]     885,120              True
├─ReLU (relu)         